In [1]:
import numpy as np

# In diesem Notebook lernen wir die Netzwerke an

1. Trainingsdatenstruktur
2. Generierung synthetischer Daten
3. Training Netzwerke

Da laut Jens Decker bei Bruker nur Randverteilungen (Summen über eine Achse) betrachtet werden, machen wir dies genauso und betrachten in jedem Netzwerk nur die Randverteilungen des Berges. Weiterhin ohne vernünftige Trainingsdaten (bzw. ohne eine wirkliche Verteilung zu erkennen), nutzen wir (unterschiedlich verteilte) synthetische Daten zum Training. Fitten die resultierenden Netzwerke am Ende gut, wurde eine passende Verteilung gewählt.

Aufgrunddessen, dass die Berge nur in Kästen separiert sind und nicht ordentlich ausgeschnitten wurden, werden Überhänge anderer Berge zu finden sein. Wir modellieren dies, in dem zufällig generierte weitere Berge außerhalb des Intervalls hinzu kommen.

Sollte dies gar nicht klappen, bleiben uns zwei Möglichkeiten:
1. Wir betrachten den Berg als ganzes und wollen dessen Verteilung herausfinden
2. Mit Hilfe eines GANs ändern wir die Daten so ab, dass der Diskriminator nicht mehr zwischen echten und synthetischen Daten unterscheiden kann. Fragestellung: Wie kriegen wir Rückfluss in die Parameter? Wahrscheinlich müssen Momente berechnet werden

Wir versuchen folgende Netzwerktypen: Feed Forward, LSTM, Transformer

## 1. Trainingsdatenstruktur
Jeder Trainingsvektor hat den höchsten Wert am Anfang stehen, darauf folgend sind die Nachbarn nach Abstand von rechts zuerst angeordnet. Ist also der höchste Punkt an 3. Stelle, wird aus dem Berg mit x-Achse 1-8 die umverteilte Version 3-4-2-5-1-6-7-8.

Jeder Datenpunkt enthält: (corrected) intensity - Abstand (mit Vorzeichen) zu höchstem Punkt - Maske

Da in einem Trainingssample zu wenig Punkte sein können, maskieren wir, ob der Datenpunkt genutzt wird (1) oder ob es zu zero-padding kam (0).

Außerhalb von LSTMs verwenden wir natürlich eine fixe Vektorlänge, welche aufgrund unserer Ordnung regulierend eingreifen sollte (da nur Tails weggeschnitten werden)

In [2]:
def input_vector_to_training_vector(input_x, input_y, input_length = 500, flatten = False):
    # Init: Nullvektoren und Paddingmaske bilden
    raw_length = max(len(input_x), input_length)
    intensities = np.zeros(raw_length)
    abstaende = np.zeros(raw_length)
    padding_mask = np.zeros(raw_length)

    padding_mask[0:len(input_x)] = 1

    # 1. Ort des größten Wertes finden
    max_index = np.argmax(input_y)
    input_max = input_x[max_index]

    # 2. Einsortieren der restlichen Punkte und Berechnung der Abstände
    intensities[0] = input_y[max_index]
    abstaende[0] = 0

    if max_index == 0:
        intensities[1:len(input_x)] = input_y[1:]
        abstaende[1:len(input_x)] = input_x[1:] - input_max
    elif max_index < len(input_x) - max_index: # rechts von max_index sind mehr Punkte
        # rechte Seite
        intensities[1:2*max_index:2] = input_y[max_index+1:2*max_index+1]
        abstaende[1:2*max_index:2] = input_x[max_index+1:2*max_index+1] - input_max
        # Tail
        intensities[2*max_index+1:len(input_x)] = input_y[2*max_index+1:]
        abstaende[2*max_index+1:len(input_x)] = input_x[2*max_index+1:] - input_max
        # linke Seite
        intensities[2:2*max_index+1:2] = input_y[max_index-1::-1]
        abstaende[2:2*max_index+1:2] = input_x[max_index-1::-1] - input_max
    else: # links von max_index sind mehr Punkte
        # rechte Seite
        intensities[1:2*(len(input_x)-max_index)-1:2] = input_y[max_index+1:]
        abstaende[1:2*(len(input_x)-max_index)-1:2] = input_x[max_index+1:] - input_max
        # linke Seite
        intensities[2:2*(len(input_x)-max_index):2] = input_y[max_index-1:2*max_index-len(input_x):-1]
        abstaende[2:2*(len(input_x)-max_index):2] = input_x[max_index-1:2*max_index-len(input_x):-1] - input_max
        # Tail
        intensities[2*(len(input_x)-max_index)-1:len(input_x)] = input_y[2*max_index-len(input_x)::-1]
        abstaende[2*(len(input_x)-max_index)-1:len(input_x)] = input_x[2*max_index-len(input_x)::-1] - input_max
    
    # 3. Ausgabevektor erstellen
    if input_length < raw_length:
        intensities = intensities[0:input_length]
        abstaende = abstaende[0:input_length]
        padding_mask = padding_mask[0:input_length]
        
    if flatten:
        output_vector = np.concatenate((intensities, abstaende, padding_mask))
    else:
        output_vector = [intensities, abstaende, padding_mask]

    return output_vector

## 2. Generierung synthetischer Daten

1. Genutzte Verteilungen
2. Einlesen von Peakdaten aus der Datenbank
3. Sample Funktion in den Punkten eines Peaks

### 1. Genutzte Verteilungen

#### Beobachtungen von Decker

Normalverteilung

In [3]:
from scipy.stats import norm

def normal(x, height, offset, mu, sigma, *kwargs):
    return offset + height*norm.pdf(x, loc=mu, scale=sigma)

Exponentially modified Gaussian distribution

In [4]:
from scipy.stats import exponnorm

def exp_normal(x, height, offset, mu, sigma, _lambda, *kwargs):
    K = 1/(sigma*_lambda)
    return offset + height*exponnorm.pdf(x, K, loc=mu, scale=sigma)

#### Allgemeinere adaptive Verteilungen

Pearsonverteilung

In [5]:
from scipy.stats import pearson3

def pearson(x, height, offset, mu, sigma, skew, *kwargs):
    return offset + height*pearson3.pdf(x, skew, loc=mu, scale=sigma)

Johnsons $S_U$-Verteilung

In [6]:
from scipy.stats import johnsonsu

def johnson(x, height, offset, mu, sigma, a, b, *kwargs):
    return offset + height*johnsonsu.pdf(x, a, b, loc=mu, scale=sigma)

Metalog-Verteilung

In [7]:
def metalog(x, height, offset, a, bounds=(-np.inf, np.inf), *kwargs):
    lower_bound, upper_bound = bounds

    # Skalierung für beschränkte Verteilungen
    if np.isfinite(lower_bound) and np.isfinite(upper_bound):
        y = (x - lower_bound) / (upper_bound - lower_bound)
    else:
        y = x

    # Berechnung der logit-Transformation
    z = np.log(y / (1 - y)) if (np.isfinite(lower_bound) and np.isfinite(upper_bound)) else y

    # Berechnung der Metalog-CDF
    cdf = a[0] + a[1] * z + a[2] * z**2 + a[3] * z**3 if len(a) >= 4 else 0
    for i in range(4, len(a)):
        cdf += a[i] * z**i

    # Berechnung der Ableitung (PDF)
    deriv_z = a[1] + 2 * a[2] * z + 3 * a[3] * z**2
    for i in range(4, len(a)):
        deriv_z += i * a[i] * z**(i - 1)

    if np.isfinite(lower_bound) and np.isfinite(upper_bound):
        pdf = deriv_z * (1 / ((y * (1 - y)) * (upper_bound - lower_bound)))
    else:
        pdf = deriv_z

    return offset + height*pdf


In [8]:
def get_distribution(name):
    return {
        'normal': normal,
        'exp_normal': exp_normal,
        'pearson': pearson,
        'johnson': johnson,
        'metalog': metalog
    }[name]

### 2. Einlesen von Peakdaten aus der Datenbank

In [10]:
from sqlalchemy import create_engine, text
from sqlalchemy_utils import database_exists, create_database

name_dataset = "testdata"

engine = create_engine('sqlite:///../Databases/%s.db' % name_dataset) # drei Backslash für relativen Pfad

# get all peak_region_number
with engine.connect() as conn:
    try:
        peak_region_number = conn.execute(text("SELECT DISTINCT peak_region FROM %s" % name_dataset)).fetchall()
        peak_region_number = [x[0] for x in peak_region_number]
    except:
        print("Keine Verbindung möglich. Datenbank existiert nicht oder ist leer.")

In [11]:
import pandas as pd

def get_peak_region(peak_region_number):
    with engine.connect() as conn:
        try:
            peak_region = conn.execute(text("SELECT * FROM %s WHERE peak_region = %d" % (name_dataset, peak_region_number))).fetchall()
            peak_region = [x for x in peak_region]
        except:
            print("Keine Verbindung möglich. Datenbank existiert nicht oder ist leer.")
    return peak_region

In [12]:
def convert_peak_region_to_df(peak_region):
    peak_region_array = np.asarray(peak_region)
    peak_region_df = pd.DataFrame(peak_region_array, columns = ["index","rt_values", "frame_indices", "mz_values", "tof_indices", "mobility_values", "scan_indices", "intensity_values", "corrected_intensity_values", "peak_region"])
    return peak_region_df

### 3. Sample Funktion in den Punkten eines Peaks

In [ ]:
def sample_synthetic_data(size, distribution, axis, neighbours = False, noise = 0.0, flatten = True):
    # Generierung von zufälligen Parametern
    height = 100*np.random.rand()
    offset = 1e6*np.random.rand()
    if distribution == "normal":
        mu = 1000*np.random.rand()
        sigma = 1000*np.random.rand()
        parameters = [height, offset, mu, sigma]
    elif distribution == "exp_normal":
        mu = 1000*np.random.rand()
        sigma = 1000*np.random.rand()
        _lambda = 10*np.random.rand()
        parameters = [height, offset, mu, sigma, _lambda]
    elif distribution == "pearson":
        mu = 1000*np.random.rand()
        sigma = 1000*np.random.rand()
        skew = 10*np.random.rand()
        parameters = [height, offset, mu, sigma, skew]
    elif distribution == "johnson":
        mu = 1000*np.random.rand()
        sigma = 1000*np.random.rand()
        a = 10*np.random.rand()
        b = 10*np.random.rand()
        parameters = [height, offset, mu, sigma, a, b]
    elif distribution == "metalog":
        a = [10*np.random.rand() for i in range(10)]
        parameters = [height, offset,*a]
    else:
        print("Verteilung nicht bekannt.")
        
    # Generierung von x-Werten
    peak_region = get_peak_region(np.random.choice(peak_region_number))
    peak_region_df = convert_peak_region_to_df(peak_region)
    x = peak_region_df[axis].values

    if neighbours:
        # TODO: Störung hinzufügen

    # Generierung von Intensity-Werten
    distribution = get_distribution(distribution)
    y = distribution(x, *parameters) # evaluate density function at the x of the peak region (x: e.g. mz_values in peak_region)
    y = y + noise*np.random.randn(len(y)) # random noise auf die Trainingsdaten legen
    trainings_vector = input_vector_to_training_vector(x, y, input_length = size, flatten = flatten)
    
    return trainings_vector, parameters

In [27]:
sample_synthetic_data(500, "normal", "mz_values")

(array([1.31998393e+04, 1.31998393e+04, 1.31998391e+04, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]),
 [64.77273856520159,
  13199.785925433805,
  450.0704270569388,
  484.34962896743326])

## 3. Training der Netzwerke

Wir trainieren in den folgenden Zellen ein Feed-Forward-Netzwerk (natürlicher Kandidat), ein LSTM und ein Transformer mit nachgeschaltetem Feed-Forward-Netzwerk. Hauptfokus liegt aber erst einmal auf das Feed-Forward-Netzwerk.

In [28]:
import os
import tensorflow as tf

In [ ]:
def get_sample(input_length = 500):
    return sample_synthetic_data(input_length, "normal", "mz_values")

Simple Feedforward model

In [ ]:
input_length = 500
trainings_vector, parameters = get_sample(input_length)
# Preprocessing


model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(input_length,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(len(parameters))
])

AttributeError: module 'tensorflow' has no attribute 'keras'

In [30]:
loss_object = tf.keras.losses.MeanSquaredError()

def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)

AttributeError: module 'tensorflow' has no attribute 'keras'

In [31]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

AttributeError: module 'tensorflow' has no attribute 'keras'

In [ ]:
# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()

  # Training loop - using batches of 32
  for _ in range(32):
    # Get training sample
    x,y = get_sample(input_length)
    # Preprocess

    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
 
  # End epoch
  train_loss_results.append(epoch_loss_avg.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f},".format(epoch, epoch_loss_avg.result()))

In [ ]:
keras_model_path = '../Networks/feedforward.keras'
model.save(keras_model_path)